In [1]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import os

In [2]:
current_dir = os.getcwd()

In [3]:
current_dir

'C:\\Users\\wavac\\Documents\\GitHub\\bangkit-final-JKT2-E'

In [4]:
TRAINING_DIR = current_dir+"\\dataset\\asl_alphabet_train\\asl_alphabet_train\\"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.2)

# VALIDATION_DIR = current_dir+"\\dataset\\asl_alphabet_test\\asl_alphabet_test\\"
# validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126,
    subset='training'
)

validation_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126,
    subset='validation'
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [5]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(29, activation='softmax')
])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [6]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=25, steps_per_epoch=271, validation_data = validation_generator, verbose = 1, validation_steps=3)

model.save(current_dir+"\\model.h5")

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 271 steps, validate for 3 steps
Epoch 1/25
271/271 [==============================] - 230s 850ms/step - loss: 3.0746 - accuracy: 0.1249 - val_loss: 2.9603 - val_accuracy: 0.1772
Epoch 2/25
271/271 [==============================] - 214s 791ms/step - loss: 2.2328 - accuracy: 0.3316 - val_loss: 2.1972 - val_accuracy: 0.3571
Epoch 3/25
271/271 [==============================] - 208s 767ms/step - loss: 1.6798 - accuracy: 0.4792 - val_loss: 2.0707 - val_accuracy: 0.3810
Epoch 4/25
271/271 [==============================] - 207s 765ms/step - loss: 1.3090 - accuracy: 0.5805 - val_loss: 1.3900 - val_accuracy: 0.5556
Epoch 5/25
271/271 [==============================] - 203s 748ms/step - loss: 1.0468 - accuracy: 0.6577 - val_loss: 1.3529 - val_accuracy: 0.5582
Epoch 6/25
271/271 [==============================] - 202s 747ms/step - loss: 0.8784 - accuracy: 0.7128 - val_loss: 1.1427 - val_accuracy: 0.6111
Epoch 7/25
271/271 [============

In [7]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

ModuleNotFoundError: No module named 'matplotlib'